In [1]:
from llama_index.core import SQLDatabase
# cargar variables de entorno
from dotenv import load_dotenv
import os
load_dotenv()

from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

# Cambia estos valores según tu configuración
usuario = os.getenv("USERNAME") # "tu_usuario"
contraseña = os.getenv("PASSWORD") # "tu_contraseña"
host = os.getenv("HOST") # "localhost"
puerto = os.getenv("PORT") # "3306"
nombre_bd = os.getenv("DATABASE") # "tu_base_de_datos"

# Conexión a MySQL
engine = create_engine(f"mysql+mysqlconnector://{usuario}:{contraseña}@{host}:{puerto}/{nombre_bd}")
sql_database = SQLDatabase(engine)

In [2]:
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.tools import QueryEngineTool
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings

Settings.llm = Ollama(model="llama3.2", request_timeout=3000.0)
Settings.embed_model = OptimumEmbedding(folder_name="./minilml6v2_onnx")

documents = SimpleDirectoryReader("./sql").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

budget_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="sql_manual_buget_tool",
    description="A RAG engine with information about how do queries in the database",
)

table_schema = {
    "pnrp.bitacora_ac": {
        "columns": {
            "ID": {"type": "int", "nullable": False, "primary_key": True, "auto_increment": True},
            "TICKET": {"type": "varchar(255)", "nullable": True},
            "CLAVE": {"type": "varchar(45)", "nullable": False},
            "MEDIDOR": {"type": "varchar(45)", "nullable": False},
            "MARCA": {"type": "varchar(45)", "nullable": True},
            "PERIODO_INICIAL": {"type": "datetime", "nullable": True},
            "PERIODO_FINAL": {"type": "datetime", "nullable": True},
            "NOMBRE": {"type": "varchar(100)", "nullable": True},
            "ANALISTA": {"type": "varchar(45)", "nullable": True},
            "FECHA_ASIGNACION": {"type": "datetime", "nullable": True},
            "FECHA_ANALISIS": {"type": "datetime", "nullable": True},
            "FECHA_REGISTRO": {"type": "datetime", "nullable": True},
            "DIAS_ULT_REVISION": {"type": "int", "nullable": True},
            "ALARMA": {"type": "tinytext", "nullable": True},
            "FECHA_ALARMA": {"type": "datetime", "nullable": True},
            "NO_CONFORMIDAD": {"type": "tinytext", "nullable": True},
            "REQUIERE_OS": {"type": "tinyint(1)", "nullable": True},
            "COMENTARIO_ANALISTA": {"type": "text", "nullable": True},
            "ES_RECURRENTE": {"type": "tinyint(1)", "nullable": True},
            "APROBACION_ANALISIS": {"type": "tinytext", "nullable": True},
            "COMENTARIO_R5": {"type": "text", "nullable": True},
            "OS": {"type": "int", "nullable": True},
            "FECHA_CREACION_OS": {"type": "datetime", "nullable": True},
            "ANOMALIA": {"type": "tinytext", "nullable": True},
            "FECHA_ANOMALIA_INICIAL": {"type": "datetime", "nullable": True},
            "FECHA_ANOMALIA_FINAL": {"type": "datetime", "nullable": True},
            "CAMBIO_MEDIDOR": {"type": "tinytext", "nullable": True},
            "SERIE_NUEVO_MEDIDOR": {"type": "bigint", "nullable": True},
            "COMENTARIO_CENTRAL": {"type": "text", "nullable": True},
            "DIRECCION": {"type": "tinytext", "nullable": True},
            "MULTIPLICADOR": {"type": "tinytext", "nullable": True},
            "MERCADO": {"type": "tinytext", "nullable": True},
            "CONSUMO_PROMEDIO": {"type": "tinytext", "nullable": True},
            "TARIFA": {"type": "tinytext", "nullable": True},
            "SECTOR": {"type": "tinytext", "nullable": True},
            "MEDIDA": {"type": "tinytext", "nullable": True},
            "LATITUD": {"type": "tinytext", "nullable": True},
            "LONGITUD": {"type": "tinytext", "nullable": True},
            "CIRCUITO": {"type": "tinytext", "nullable": True},
            "CRITICIDAD_ALARMA": {"type": "varchar(255)", "nullable": True},
            "PONDERACION_CRITICIDAD": {"type": "decimal(3,2)", "nullable": True},
            "ESTADO": {"type": "tinytext", "nullable": True},
            "REGION_ASIGNACION": {"type": "tinytext", "nullable": True},
        }
    }
}


sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["pnrp.bitacora_ac"],
    verbose=True,
    table_schema=table_schema,
)
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_tool",
    description=(
        "Useful for translating a natural language query into a SQL query"
    ),
)



c:\Users\Auxiliar\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from llama_index.core.agent import ReActAgent
agent = ReActAgent.from_tools(
    [budget_tool, sql_tool], 
    llm=Settings.llm, 
    verbose=True,
    max_iterations=30,
)

In [4]:
instructions = """
Siempre consulta el manual antes de realizar consultas sql asi tendras una idea de como hacerlo.
"""
input = "Cuál es la información del ticket TK-ELS-1"

response = agent.chat(f"""
Instruccion: {instructions}. 
                      
Pregunta del usuario {input}
""")

print(response)

> Running step ac903cd4-b168-4059-a8ae-5f66b4dd6220. Step input: 
Instruccion: 
Siempre consulta el manual antes de realizar consultas sql asi tendras una idea de como hacerlo.
. 
                      
Pregunta del usuario Cuál es la información del ticket TK-ELS-1

Thought: The current language of the user is: español. I need to use a tool to help me answer the question.
Action: sql_manual_buget_tool
Action Input: {'properties': AttributedDict([('input', AttributedDict([('title', 'Input'), ('type', 'string')]))]), 'required': ['input'], 'type': 'object'}
Observation: SELECT CLAVE, MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '12345' ORDER BY FECHA DESC LIMIT 20;
> Running step 8cf27979-7e7c-4b77-8ada-c6ffeb0e7b3f. Step input: None
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step 9bbfdc77-5fa7-462f-b903-938b6395bc70. Step input: None
Thought: The current language of the user 